In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob,os
import sys
import scipy
from importlib import  reload
from time import process_time 
#from libraries.lib_gather_data import get_hhid_FIES
from datetime import datetime
%load_ext autoreload

In [2]:
#from shock_libraries import *
#from plotting_libraries import *
#from response_libraries import get_response_sp
#
from income_shock_libraries_ps import *
#
#from libraries.lib_country_dir import set_directories, load_survey_data, get_places_dict
#from libraries.lib_get_hh_savings import get_hh_savings
#from libraries.pandas_helper import broadcast_simple

In [3]:
from income_shock_libraries_ps import *

In [4]:
# formatting & aesthetics
font = {'family':'sans serif', 'size':10}
plt.rc('font', **font)
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.facecolor'] = 'white'
sns.set_style("white")

sns_pal = sns.color_palette('Set1', n_colors=8, desat=.4)
greys_pal = sns.color_palette('Greys', n_colors=9)

In [5]:
# lol = pd.read_csv('./2015FIES/LFSJul2015_merge.csv')
# lol

In [6]:
def entre_shock(s_sector='LFS_sector',selector_type = 0):
    """
    """
    #s_sector = 'LFS_sector' #### <HARDCODE> testing

    #mr = merge_rank()
    #mr = merge_rank(rank_file='./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv',labor_file='./csv/ph_labor_force.csv',outfile='./temp/_labor_rank_merge_test.csv',merge_col='a09_pqkb')
    mr = merge_rank(rank_file='./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv',labor_file='./csv/ph_labor_force.csv',outfile='./temp/_labor_rank_merge_test.csv',merge_col='a09_pqkb')



    if not 'LFS_sector' in mr.columns:
        mr = mr.rename(columns={'LFS_sector_y': 'LFS_sector'})
        # get subset: a09_pqkb

    if 'hhid_lfs' in mr.columns:
        mr_subset = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home','E_sector','pwgt']]
    else:
        mr_subset = mr[['cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home','E_sector','pwgt']]


    mr_subset
    indexNames = mr_subset[mr_subset['a09_pqkb'] == 'nan' ].index
    # Delete these row indexes from dataFrame
    mr_subset.drop(indexNames , inplace=True)
    mr_subset = mr_subset.reset_index(drop=True)
    # get subset: c19_pclass

    indexNames2 = mr_subset[mr_subset['c19_pclass'] == 'nan' ].index

    # Delete these row indexes from dataFrame
    mr_subset.drop(indexNames2 , inplace=True)
    mr_subset = mr_subset.reset_index(drop=True)

    # make new column of combined string a09 && c19:
    mr_subset['a09c19'] = mr_subset['a09_pqkb'] +'-'+mr_subset['c19_pclass']

    # enforce string:
    mr_subset['a09_pqkb'] = [str(q).strip() for q in mr_subset['a09_pqkb']] # enforce type = string
    mr_subset['LFS_sector'] = [str(q).strip() for q in mr_subset['LFS_sector']] # enforce type = string
    mr_subset['c19_pclass'] = [str(q).strip() for q in mr_subset['c19_pclass']] # enforce type = string
    mr_subset['a09c19'] = [str(q).strip() for q in mr_subset['a09c19']] # enforce type = string
    if 'E_sector' in mr_subset.columns:
        mr_subset['E_sector'] = [str(q).strip() for q in mr_subset['E_sector']] # enforce type = string


    ## implement switch statement here:
    #selector_type = 0 # 0 = default, 1 = nonag_wage, 2 = entrepreneurial ####### <HARDCODE> for testing

    if selector_type == 0:
        df_select = mr_subset
    elif selector_type == 1:
        df_select = mr_subset [~mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])] # NOT entrepreneurial income
    elif selector_type == 2:
        df_select = mr_subset [mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])] # YES entrepreneurial income
    else:
        print('df_selector error')

    # generate counts by subsector (a09..) and sectors (s_sector)
    df_select['sector_count'] = df_select.groupby([s_sector])['pwgt'].transform('sum') #count total unique sectors and append to mr_subset
    df_select['desc_count'] = df_select.groupby([s_sector,'a09_pqkb'])['pwgt'].transform('sum') #count total unique sectors and append to mr_subset

    df_select['sector_frac'] = df_select['desc_count'] / df_select['sector_count'] # get fraction of sector as weightin

    dftest = df_select

    # drop duplicates (now that overall weighting established)
    df_select = df_select.drop_duplicates(subset='a09_pqkb')
    df_select = df_select.reset_index(drop=True)

    # generate probability and combine with relative weighting
    df_select['partial_prob'] = np.nan
    df_select['third_col'] = np.nan
    df_select['dummy'] = np.nan

        # incorporate Kayenat tables into 'di' &&
        # nested logic to incorporate 0-4 scale for social distancing measures
        ## where scores of 0 & 1 result in complete job lost, due to unable to distance
    i=0
    while i < len(df_select):

        if df_select.demand_scale[i] == 0:

            # incorporate 0-4 scale logic:

            if df_select.w_home[i] == 0:
                df_select.partial_prob[i] = 0

            elif df_select.w_home[i] == 1:
                df_select.partial_prob[i] = 0

            else:
                df_select.partial_prob[i] = df_select.sector_frac[i] * (random.randint(0,50)/100)


        elif df_select.demand_scale[i] == 0.5: 

            # incorporate 0-4 scale logic:
            if df_select.w_home[i] == 0:
                df_select.partial_prob[i] = 0

            elif df_select.w_home[i] == 1:
                df_select.partial_prob[i] = 0

            else: 
                df_select.partial_prob[i] = df_select.sector_frac[i] * (random.randint(50,100)/100)

        elif df_select.demand_scale[i] == 1.0:
            df_select.partial_prob[i] = df_select.sector_frac[i]
        else:
            df_select.dummy[i] = -99


    # incorporate 3rd column modifiers here:
        if (df_select['c19_pclass'][i] == "Gov't/Gov't Corporation"):
            df_select.partial_prob[i] = df_select.sector_frac[i]  # essentially reverts the random uniform logic implemented above


        i = i + 1 
        # remove nans in summing fields, and dummy storage
    del df_select['dummy']




    # section--> get mean probability by sector:

    #storage['fa'] = np.nan
    storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_select[s_sector])])
    storage.index.names = ['sector']

    # iterate through loop of sector names
    for seclist in np.unique(df_select[s_sector]):
        pillow = 1 - (df_select[df_select[s_sector] == seclist].partial_prob.sum()) # KEY: logic inverted here-- now a probability of 1- change job loss = 'fraction affected'

        storage.loc[seclist,'fa'] = pillow # pillow is just an intermediate storage variable
        storage.loc[seclist,'di'] = 1

       # print('seclist ='+ seclist)

    storage.loc['government','fa'] = 0  ### ENFORCE ULTIMATE ZERO FOR GOVT!

    tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    storage.to_csv('./temp/table_full_'+s_sector+'_' +tstamp+'.csv')

    return(storage, selector_type)
#storage 

In [7]:
def generate_shock_100_entre(selector_type):  # initialize shock sector storage dataframe
   
    '''
   current hard coding for sensitivity analysis, 20200413: requires cleaning for further implementation
   - addition of modularity
   - 
   - current functionality:
       - outputs csv to location: './temp/sect_iter_100.csv
       - containing data frame with 101 simulations of <rand_weighted_shock_distance():
    - runtime: ~10minutes
   '''

    stor,selector_type = entre_shock()
    del stor['di']

    # set number of iterations
    p = 0
    n_iter = 99

    # model and store stochastic sector response
    while p < n_iter:
        
        new_val,selector_type = entre_shock()
        del new_val['di']
        new_val = new_val.rename(columns={'fa': ('iter'+str(p))})

        # pd.merge(labor,rank, on=merge_col, how='left')
        stor = pd.merge(stor,new_val,on='sector', how='left')
        p = p+ 1
        print(p)
    
    ## implement switch statement here:
    #selector_type = 0 # 0 = default, 1 = nonag_wage, 2 = entrepreneurial ####### <HARDCODE> for testing
    #selector_type = 0
    if selector_type == 0:
        type_out = 'fullsect'
    elif selector_type == 1:
        type_out = 'nonag-wage'
    elif selector_type == 2:
        type_out = 'entre'

    else:
        print('df_selector error')

    tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    mc_output_filename = str('./temp/_shock_mc_'+ type_out+'_'+tstamp+ '.csv')


    stor.to_csv(mc_output_filename)
    return (mc_output_filename)

In [8]:
def get_shock_stats_entre(input_file):
    # generate shock table statistics
    #df['mean'] = df.mean(axis=1)

    # load csv to dataframe:
    #dfs = pd.read_csv('./temp/sect_iter_100.csv') # original
    dfs = pd.read_csv(input_file) # modified 20200420
    # set index to LFS_sector
    dfs.set_index('sector')

    # compute statistics:
    dfs['mean'] = dfs.mean(axis=1)
    #print(dfs['mean'])
    dfs['std_dev'] = dfs.std(axis=1)
    #print(dfs['std_dev'])

    #round to 3 dec:
    dfs['mean'] = [(round(q, 3)) for q in dfs['mean']]
    dfs['std_dev'] = [(round(q, 3)) for q in dfs['std_dev']]

    # new datafame storing just info:
    df_stat = dfs[['sector','mean','std_dev']].set_index('sector')
    df_stat
    # df_stat.to_csv('./temp/phi_get_shock_input.csv') # original
    tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    df_stat.to_csv('./temp/phi_'+input_file[7:]) # modified 20200420
    
    
    
    
    return(df_stat)


In [10]:
set_option = 1


In [11]:
xx, sel_type = entre_shock(selector_type =set_option)
xx

sel_type

The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv


/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

1

In [12]:
sel_type

1

In [13]:
file_sto = generate_shock_100_entre(selector_type= set_option)

The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv


/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-packages/ipykernel_launcher.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/geospatial/lib/python3.6/site-

The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
1
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
2
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
3
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
4
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
5
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
6
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
7
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
8
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
9
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
10
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
11
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
12
The_rank_file =./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv
13
The_rank_file =./temp

In [14]:
final = get_shock_stats_entre(file_sto)
final 

,mean,std_dev
sector,,
ag,0.017,0.002
construction,0.760,0.122
eduhealth,0.358,0.025
extraction,0.744,0.094
finance,0.098,0.026
food_entertainment,0.976,0.004
government,0.000,0.000
information,0.047,0.000
manufacturing,0.769,0.009
